In [1]:
import numpy as np
import mxnet as mx
from mxnet import gluon,nd,autograd
from mxnet.gluon import nn

In [2]:
def f_transform(data):
    return nd.moveaxis(data,2,0).astype('float32')/255

In [3]:
tD = mx.gluon.data.vision.MNIST(train=True).transform_first(f_transform)
vD = mx.gluon.data.vision.MNIST(train=False).transform_first(f_transform)

In [4]:
bs = 50
tL = mx.gluon.data.DataLoader(tD,shuffle=True,batch_size=bs)
vL = mx.gluon.data.DataLoader(vD,shuffle=True,batch_size=bs)


In [5]:
model = nn.Sequential()
with model.name_scope():
    model.add(
        nn.Conv2D(channels=32,kernel_size=(5,5),activation='relu'),
        nn.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        nn.Dropout(0.3),
        nn.Conv2D(channels=64,kernel_size=(5,5),activation='relu'),
        nn.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        nn.Flatten(),
        nn.Dense(128,activation='relu'),
        nn.Dense(10,activation=None),
    )

In [7]:
model.initialize(mx.init.Xavier())

In [8]:
trainer = gluon.Trainer(
    params = model.collect_params(),
    optimizer = 'sgd',
    optimizer_params = {'learning_rate':0.01},
)

In [9]:
metric = mx.metric.Accuracy()
loss_function = mx.gluon.loss.SoftmaxCrossEntropyLoss()

In [10]:
ctx = mx.cpu(0)

In [11]:
e = 5
for ep in range(e):
    for X,y in tL:
        X = X.as_in_context(ctx)
        y = y.as_in_context(ctx)
        with autograd.record():
            y_hat = model(X)
            loss = loss_function(y_hat,y)
        loss.backward()
        metric.update(y,y_hat)
        trainer.step(batch_size=X.shape[0])
    name,acc = metric.get()
    print(acc)
    metric.reset()

0.8155833333333333
0.9400333333333334
0.9588166666666667
0.9670666666666666
0.9719


In [13]:
metric = mx.metric.Accuracy()
for Xv,yv in vL:
    Xv = Xv.as_in_context(ctx)
    yv = yv.as_in_context(ctx)
    metric.update(yv,model(Xv))
name,acc = metric.get()
print(acc)

0.9796
